In [1]:
from os.path import join
import os
import re

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2


device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/인공지능프로젝트/PLSU_4064_classification용.zip" -d "/content/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/PLSU_4064/Mask/mask2512.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask2512.jpg  
  inflating: /content/PLSU_4064/Mask/mask255.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask255.jpg  
  inflating: /content/PLSU_4064/Mask/mask533.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask533.jpg  
  inflating: /content/PLSU_4064/Mask/mask2274.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask2274.jpg  
  inflating: /content/PLSU_4064/Mask/mask3181.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask3181.jpg  
  inflating: /content/PLSU_4064/Mask/mask1796.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask1796.jpg  
  inflating: /content/PLSU_4064/Mask/mask1782.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask1782.jpg  
  inflating: /content/PLSU_4064/Mask/mask19.jpg  
  inflating: /content/__MACOSX/PLSU_4064/Mask/._mask19.jpg  
  inflating: /content/PLSU_4064/Mask/mask3195.jpg  


# Sorting Data

In [4]:
## 파일 이름에서 숫자 부분을 추출하는 함수
def extract_number(file_name):
    numbers = re.findall(r'\d+', file_name)
    return int(numbers[0]) if numbers else 0


def list_files_in_directory(directory_path):
    items_in_directory = sorted(os.listdir(directory_path))
    files_in_directory = [item for item in items_in_directory if os.path.isfile(os.path.join(directory_path, item))]

    return files_in_directory

In [5]:
mask_directory = '/content/PLSU_4064/Mask'
mask_files = list_files_in_directory(mask_directory)
sorted_mask_files = sorted(mask_files, key=extract_number)
mask_paths = [os.path.join(mask_directory, file_name) for file_name in sorted_mask_files]

# 이미지와 마스크의 경로 예시를 보고 혹시 파일 리스트가 순서대로 정렬되어 있지 않는지 확인해봅니다.
print(sorted_mask_files)
print(len(sorted_mask_files))
print(f'마스크 경로 예시 : {mask_paths[:5]}')

['mask1.jpg', 'mask2.jpg', 'mask3.jpg', 'mask4.jpg', 'mask5.jpg', 'mask6.jpg', 'mask7.jpg', 'mask8.jpg', 'mask9.jpg', 'mask10.jpg', 'mask11.jpg', 'mask12.jpg', 'mask13.jpg', 'mask14.jpg', 'mask15.jpg', 'mask16.jpg', 'mask17.jpg', 'mask18.jpg', 'mask19.jpg', 'mask20.jpg', 'mask21.jpg', 'mask22.jpg', 'mask23.jpg', 'mask24.jpg', 'mask25.jpg', 'mask26.jpg', 'mask27.jpg', 'mask28.jpg', 'mask29.jpg', 'mask30.jpg', 'mask31.jpg', 'mask32.jpg', 'mask33.jpg', 'mask34.jpg', 'mask35.jpg', 'mask36.jpg', 'mask37.jpg', 'mask38.jpg', 'mask39.jpg', 'mask40.jpg', 'mask41.jpg', 'mask42.jpg', 'mask43.jpg', 'mask44.jpg', 'mask45.jpg', 'mask46.jpg', 'mask47.jpg', 'mask48.jpg', 'mask49.jpg', 'mask50.jpg', 'mask51.jpg', 'mask52.jpg', 'mask53.jpg', 'mask54.jpg', 'mask55.jpg', 'mask56.jpg', 'mask57.jpg', 'mask58.jpg', 'mask59.jpg', 'mask60.jpg', 'mask61.jpg', 'mask62.jpg', 'mask63.jpg', 'mask64.jpg', 'mask65.jpg', 'mask66.jpg', 'mask67.jpg', 'mask68.jpg', 'mask69.jpg', 'mask70.jpg', 'mask71.jpg', 'mask72.jpg', 

# Classification DataSet

In [6]:
import torch
import cv2
from torch import nn, optim
from torchvision.transforms.functional import to_tensor
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
class CustomDataset(Dataset):
    def __init__(self, mask_paths, labels, size=(256, 256), mode='train'):

        self.labels = labels
        self.mask_paths = mask_paths
        self.resize = size
        self.mode = mode

    def __len__(self):
        return len(self.mask_paths)

    def __getitem__(self, idx):

      mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)

      # 마스크의 크기를 256x256으로 변경
      mask = cv2.resize(mask, self.resize)

      # 마스크를 [0, 1] 범위로 정규화
      mask = mask.astype(np.float32) / 255
      # 마스크를 PyTorch 텐서로 변환
      mask = to_tensor(mask)
      label = self.labels[idx]
      label = torch.tensor(label, dtype=torch.long)
      return mask, label


def load_labels(csv_file):
    data = pd.read_csv(csv_file)
    #이상한 라벨 삭제
    incorrect_labels = data['label'].astype(str).str.contains(r'[0-9]-[0-9]')
    filtered_data = data[~incorrect_labels]

    # 라벨을 정수로 매핑
    label_mapping = {
        '000': 0,
        '001': 1,
        '010': 2,
        '011': 3,
        '100': 4,
        '101': 5,
        '110': 6,
        '111': 7
    }
    labels = filtered_data['label'].apply(lambda x: label_mapping[x]).values
    return labels, filtered_data.index



# 사용할 라벨과 인덱스를 로드합니다.
labels, valid_indices = load_labels('/content/PLSU_4064/classification_label_4064.csv')

# 이제 `valid_indices`를 사용하여 `mask_paths`에서 올바른 항목만을 필터링할 수 있습니다.
valid_mask_paths = [mask_paths[i] for i in valid_indices]


# 전체 데이터셋 로드
full_dataset = CustomDataset(valid_mask_paths, labels=labels, size=(256,256))

total_size = len(full_dataset)
train_size = int(total_size * 0.8)
valid_size = int(total_size * 0.1)
test_size = total_size - train_size - valid_size

# 데이터셋 분할
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, valid_size, test_size])
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


3139
392
393


# Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
        super(PatchEmbedding, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim
        self.num_patches = (image_size // patch_size) ** 2
        self.projection = nn.Conv2d(
            in_channels=in_channels,
            out_channels=embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )

    def forward(self, x):
        x = self.projection(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x


class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, mlp_drop=0.):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(mlp_drop)

    def forward(self, x):
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C //
                                  self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        x = (q @ k.transpose(-2, -1)) * self.scale
        x = x.softmax(dim=-1)
        x = self.attn_drop(x)
        x = (x @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class EncoderBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4.0, qkv_bias=False, attn_drop=0.0, proj_drop=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.attn = Attention(dim=dim, num_heads=num_heads,
                              qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=proj_drop)
        self.mlp = MLP(in_features=dim, hidden_features=int(
            dim * mlp_ratio), out_features=dim)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


class MLPHead(nn.Module):
    def __init__(self, embed_dim, mlp_hidden_dim, num_classes):
        super(MLPHead, self).__init__()
        self.embed_dim = embed_dim
        self.mlp_hidden_dim = mlp_hidden_dim
        self.num_classes = num_classes
        self.fc1 = nn.Linear(embed_dim, mlp_hidden_dim)
        self.fc2 = nn.Linear(mlp_hidden_dim, mlp_hidden_dim)
        self.fc3 = nn.Linear(mlp_hidden_dim, num_classes)
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x


class ViT(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, num_classes, embed_dim, depth, num_heads, mlp_ratio=4.0, qkv_bias=False, attn_drop=0.0, proj_drop=0.0):
        super().__init__()
        self.patch_embed = PatchEmbedding(
            image_size=image_size, patch_size=patch_size, in_channels=in_channels, embed_dim=embed_dim)
        self.num_patches = self.patch_embed.num_patches
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=proj_drop)
        self.encoder_blocks = nn.ModuleList([
            EncoderBlock(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio,
                         qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=proj_drop)
            for _ in range(depth)
        ])
        self.mlp_head = MLPHead(
            embed_dim=embed_dim, mlp_hidden_dim=embed_dim * 4, num_classes=num_classes)

    def forward(self, x):
        x = self.patch_embed(x)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
        x = x[:, 0]
        x = self.mlp_head(x)
        return x

In [23]:
!pip install -q -U segmentation-models-pytorch albumentations > /dev/null
import segmentation_models_pytorch as smp

ERROR: Operation cancelled by user


In [8]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
# F1 점수 계산을 위한 함수
def calculate_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')
# 정확도 계산을 위한 함수
def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Debug

In [ ]:
# import matplotlib.pyplot as plt
# # DataLoader에서 첫 번째 배치를 가져옵니다.
# data_iter = iter(train_loader)
# images, labels = next(data_iter)

# # 첫 번째 이미지를 가져와서 시각화합니다.
# # 이미지는 0에서 1 사이의 값으로 이미 정규화되어 있다고 가정합니다.
# image = images[0].squeeze()  # 마스크 이미지이므로 채널 차원을 제거합니다.

# # 이미지를 numpy 배열로 변환합니다.
# image_np = image.numpy()

# # 라벨을 출력하고 이미지를 표시합니다.
# print(f'Label: {labels[0]}')
# plt.imshow(image_np, cmap='gray')  # 회색조로 표시
# plt.show()

In [36]:
# from torchvision import models, transforms

# model = ViT(
#     image_size=256,  # 이미지 크기
#     patch_size=16,   # 패치 크기
#     in_channels=1,   # 입력 채널 (회색조 이미지는 1)
#     num_classes=8,  # 분류할 클래스 수 (예시로 10 사용)
#     embed_dim=768,   # 임베딩 차원
#     depth=12,        # 트랜스포머 블록의 수
#     num_heads=12,    # 어텐션 헤드의 수
#     mlp_ratio=4.0    # MLP 비율
# )
# model = model.to(device)

# # 손실 함수와 옵티마이저 설정
# criterion = nn.CrossEntropyLoss()

# # DataLoader에서 첫 번째 배치를 가져옵니다.
# data_iter = iter(train_loader)
# images, labels = next(data_iter)
# image = images[0].to(device)
# label = labels[0].to(device)


# # 이미지와 라벨의 크기와 타입을 출력합니다.
# print(f"Images shape: {images.shape}, dtype: {images.dtype}")
# print(f"Labels shape: {labels.shape}, dtype: {labels.dtype}")

# # 라벨의 값 범위 확인 (예: 0부터 num_classes-1까지)
# print(f"Label values: {labels.unique()}")

# # 모델의 첫 번째 레이어에 임의의 데이터를 전달하여 출력을 확인합니다.


# # 모델에 임의의 입력을 전달하고 출력 크기를 확인합니다.
# image = image.unsqueeze(0)
# label = label.unsqueeze(0)  # 라벨에도 배치 차원 추가
# output = model(image)
# print(f"Model output shape: {output.shape}")

# print(output)
# print(label)
# loss = criterion(output, label)
# print(loss)

Images shape: torch.Size([32, 1, 256, 256]), dtype: torch.float32
Labels shape: torch.Size([32]), dtype: torch.int64
Label values: tensor([0, 1, 2, 3, 4, 5, 6])
Model output shape: torch.Size([1, 8])
tensor([[-0.0617, -0.0879,  0.0496,  0.1436,  0.1796,  0.1409,  0.0830, -0.0454]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([4], device='cuda:0')
tensor(1.9547, device='cuda:0', grad_fn=<NllLossBackward0>)


# Train

In [9]:
# 각 클래스의 빈도를 계산
class_counts = torch.zeros(8) # 클래스 수에 따라 조정
for _, label in train_loader:
    class_counts += torch.bincount(label, minlength=class_counts.size(0))
# 각 클래스의 가중치를 계산
total_samples = class_counts.sum()
class_weights = total_samples / class_counts

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torchvision import models, transforms
#하이퍼 파라미터
num_epochs = 50
learning_rate = 0.001
T_max = 50  # 학습률 주기

# 성능 기록을 위한 리스트 초기화
train_losses = []
val_losses = []
f1_scores_epochs = []
iou_scores_epochs = []

# 최상의 모델을 저장하기 위한 초기 설정
best_f1_score = 0.0
early_stopping_counter = 0
patience = 10  # 성능이 개선되지 않는 에포크 한계값


# model = ViT(
#     image_size=256,  # 이미지 크기
#     patch_size=16,   # 패치 크기
#     in_channels=1,   # 입력 채널 (회색조 이미지는 1)
#     num_classes=8,  # 분류할 클래스 수 (예시로 10 사용)
#     embed_dim=768,   # 임베딩 차원
#     depth=12,        # 트랜스포머 블록의 수
#     num_heads=12,    # 어텐션 헤드의 수
#     mlp_ratio=4.0    # MLP 비율
# )
# model = model.to(device)

# 모델 정의
model = models.resnet50(pretrained=True)

# 입력 채널을 1로 변경 (회색조 이미지용)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# 분류기(classifier)를 8개 클래스 출력으로 변경
model.fc = nn.Linear(model.fc.in_features, 8)

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)




# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)

# CosineAnnealingLR 스케줄러 초기화
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=1e-6)

# 학습 루프
num_epochs = 50  # 에폭 수

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    all_labels = []
    all_preds = []

    for images, labels in tqdm(train_loader):
        images = images.float().to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    scheduler.step
    train_losses.append(epoch_loss / len(train_loader))

    # 검증 부분
    model.eval()
    val_loss = 0
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            images = images.float().to(device)
            labels = labels.to(device).long()
            outputs = model(images)
            loss = criterion(outputs, lab els)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    val_loss /= len(valid_loader)
    val_losses.append(val_loss)

    # F1 점수 계산
    train_f1 = calculate_f1_score(all_labels, all_preds)
    val_f1 = calculate_f1_score(val_labels, val_preds)
    train_accuracy = calculate_accuracy(all_labels, all_preds)
    val_accuracy = calculate_accuracy(val_labels, val_preds)
    f1_scores_epochs.append(val_f1)

    print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader)}, Validation Loss: {val_loss}, Train F-1: {train_f1}, Validation F-1: {val_f1}, Train Acc: {train_accuracy}, Val Acc: {val_accuracy}')

    # Early Stopping
    if val_f1 > best_f1_score:
        best_f1_score = val_f1
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping triggered")
            break

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 13/13 [00:02<00:00,  4.53it/s]


Epoch 1, Loss: 1.8487094953806713, Validation Loss: 2.2320045782969546, Train F-1: 0.24098870433954003, Validation F-1: 0.22009342479168056, Train Acc: 0.23861102261866837, Val Acc: 0.34183673469387754


100%|██████████| 13/13 [00:03<00:00,  4.24it/s]


Epoch 2, Loss: 1.4722398844632236, Validation Loss: 2.6247036273662863, Train F-1: 0.3753715760591489, Validation F-1: 0.10677604319204057, Train Acc: 0.3889773813316343, Val Acc: 0.16581632653061223


100%|██████████| 13/13 [00:03<00:00,  3.90it/s]


...

Epoch 34, Loss: 0.09316934696916077, Validation Loss: 1.6303828496199388, Train F-1: 0.9665250365105619, Validation F-1: 0.6160793566950143, Train Acc: 0.9665498566422428, Val Acc: 0.6275510204081632


100%|██████████| 13/13 [00:04<00:00,  3.07it/s]


Epoch 35, Loss: 0.12341030842314164, Validation Loss: 1.5593098218624408, Train F-1: 0.9582598126830029, Validation F-1: 0.6564693599108603, Train Acc: 0.9582669640012743, Val Acc: 0.6530612244897959


100%|██████████| 13/13 [00:03<00:00,  3.46it/s]

Epoch 36, Loss: 0.08543171932111786, Validation Loss: 1.645872372847337, Train F-1: 0.9620354484535141, Validation F-1: 0.6971860385910243, Train Acc: 0.9620898375278751, Val Acc: 0.6964285714285714
Early stopping triggered


In [20]:
mask = cv2.imread('/content/classification예시.jpeg', cv2.IMREAD_GRAYSCALE)
# 마스크를 [0, 1] 범위로 정규화
mask = mask.astype(np.float32) / 255
# 마스크를 PyTorch 텐서로 변환
mask = to_tensor(mask)  # PyTorch 텐서로 변환
mask = mask.unsqueeze(0)  # 배치 차원 추가
mask = mask.float().to(device)  # 디바이스(예: CUDA)로 이동
# 모델 예측
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():  # 그래디언트 계산 비활성화
    outputs = model(mask)
    _, predicted = torch.max(outputs, 1)  # 가장 높은 점수를 가진 클래스 추출

# 예측된 클래스 출력
print(f"Predicted class: {predicted.item()}")

Predicted class: 5
